In [2]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Book Assistant",
#     instructions="You help users with their question on the files they upload.",
#     model="gpt-4o-mini",
#     tools=[{"type": "file_search"}],
# )
assistant_id = "asst_2vAP8Lkhf0be1RnREnLQsZQJ"

In [3]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want you to help me with this file.",
        }
    ]
)
print(thread)

Thread(id='thread_bT4FUPMghcl7ImWKWAwNCj4n', created_at=1727410683, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [4]:
file = client.files.create(
    file=client.file_from_path("./files/chapter_one.txt"),
    purpose="assistants",
)
print(file)

FileObject(id='file-UQ8SL5EKJfbVWu7M4dUyjGNZ', bytes=35531, created_at=1727410777, filename='chapter_one.txt', object='file', purpose='assistants', status='processed', status_details=None)


In [6]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Please refer to the uploaded file for my question.",
    attachments=[
        {
            "file_id": file.id,
            "tools": [
                {
                    "type": "file_search",
                }
            ],
        }
    ],
)

Message(id='msg_R13lA5p1zMRt9IrhUsESrg51', assistant_id=None, attachments=[Attachment(file_id='file-UQ8SL5EKJfbVWu7M4dUyjGNZ', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Please refer to the uploaded file for my question.'), type='text')], created_at=1727424307, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_bT4FUPMghcl7ImWKWAwNCj4n')

In [34]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(f"Source: {annotation.file_citation}")


# def get_tool_outputs(run_id, thread_id):
#     run = get_run(run_id, thread_id)
#     outputs = []
#     for action in run.required_action.submit_tool_outputs.tool_calls:
#         action_id = action.id
#         function = action.function
#         print(f"Calling function: {function.name} with arg {function.arguments}")
#         outputs.append(
#             {
#                 "output": functions_map[function.name](json.loads(function.arguments)),
#                 "tool_call_id": action_id,
#             }
#         )
#     return outputs


# def submit_tool_outputs(run_id, thread_id):
#     outputs = get_tool_outputs(run_id, thread_id)
#     return client.beta.threads.runs.submit_tool_outputs(
#         run_id=run_id,
#         thread_id=thread_id,
#         tool_outputs=outputs,
#     )

In [21]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

print(run)

Run(id='run_Et6iu8qNsHs1HtSQCy6XBI34', assistant_id='asst_2vAP8Lkhf0be1RnREnLQsZQJ', cancelled_at=None, completed_at=None, created_at=1727429340, expires_at=1727429940, failed_at=None, incomplete_details=None, instructions='You help users with their question on the files they upload.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_bT4FUPMghcl7ImWKWAwNCj4n', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [22]:
get_run(run.id, thread.id).status

'completed'

In [35]:
get_messages(thread.id)

user: I want you to help me with this file.
user: Please refer to the uploaded file for my question.
assistant: Please specify your question about the uploaded file, and I'll do my best to assist you!
user: I want to know where does Winston live and how his apartment looks like.
assistant: Winston lives in an apartment located in a building called Victory Mansions. The description of his surroundings is quite dreary and reflects the oppressive environment in which he lives.

**Winston's Apartment and the Building:**
- **Location**: Victory Mansions, a grim, run-down building in a decaying part of London, which is the chief city of the province of Oceania.
- **Exterior**: The building, described with a facade that conveys a sense of neglect, has stairwells that often smell of boiled cabbage and old rugs. It has a poster with the face of Big Brother watching over the entrance, reinforcing the omnipresence of the Party【7:0†source】.
- **Interior**: Winston's flat is seven flights up, and t

In [19]:
send_message(
    thread.id,
    "Where does he work?",
)

Message(id='msg_BuZOd8fIoiv6YVQM7NTYnfJS', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Where does he work?'), type='text')], created_at=1727429167, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_bT4FUPMghcl7ImWKWAwNCj4n')